In [2]:
import nltk

In [7]:
nltk.download('cess_esp')

[nltk_data] Downloading package cess_esp to /home/jiwidi/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.


True

# Ejercicio 1

In [75]:
#Cargamos el corpus
from nltk.corpus import cess_esp

corpus_sentences = cess_esp.tagged_sents()
number_sentences = len(corpus_sentences)
print("number of sentnces:", number_sentences)
print("number of words:", number_sentences)


number of sentnces: 6030
number of words: 6030


In [76]:
processed_corpus = []
for sentence in corpus_sentences:
    tmp_sentence = []
    for word in sentence:
        if word[0] == "*0*":
            continue
        if word[1][0] == "v" or "F":
            new_word = (word[0], word[1][:2])
        else:
            new_word = (palabra_etiquetada[0], word[1][:1])
        tmp_sentence.append(new_word)
    processed_corpus.append(tmp_sentence)

In [77]:
#Split training and test

len_training = int(0.9*len(processed_corpus))
training_notcv = processed_corpus[:len_training]
test_notcv = processed_corpus[len_training:]

# Ejercicio 2

In [78]:
from nltk.tag import hmm
from nltk.tag import tnt
from tqdm import tqdm
#TNT
tagger_tnt = tnt.TnT()
tagger_tnt.train(training_notcv)
print("Precission for TNT without CV")
print(tagger_tnt.evaluate(test_notcv))

#HMM 
tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_notcv)
print("Precission for HMM without CV")
print(tagger_hmm.evaluate(test_notcv))

Precission for TNT without CV
0.8241774601756736
Precission for HMM without CV
0.881048086943576


# Ejercico 3

In [79]:
from random import shuffle
import random
from sklearn.model_selection import KFold
import numpy as np

random.seed(17)


#Sin barajar
results_unshuffled_tnt = []
results_unshuffled_hmm = []
kf_unshuffled = KFold(n_splits=10, shuffle=False)
for train_index, test_index in kf_unshuffled.split(processed_corpus):
    #Split data
    training_cv = np.take(processed_corpus,train_index)
    test_cv = np.take(processed_corpus,test_index)
    #TNT
    tagger_tnt = tnt.TnT()
    tagger_tnt.train(training_cv)
    results_unshuffled_tnt.append(tagger_tnt.evaluate(test_cv))
    #HMM
    tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_notcv)
    results_unshuffled_hmm.append(tagger_hmm.evaluate(test_cv))


#Barajando
kf_shuffled = KFold(n_splits=10, shuffle=True)
results_shuffled_tnt = []
results_shuffled_hmm = []
for train_index, test_index in kf_shuffled.split(processed_corpus):
    #Split data
    training_cv = np.take(processed_corpus,train_index)
    test_cv = np.take(processed_corpus,test_index)
    #TNT
    tagger_tnt = tnt.TnT()
    tagger_tnt.train(training_cv)
    results_shuffled_tnt.append(tagger_tnt.evaluate(test_cv))
    #HMM
    tagger_hmm = hmm.HiddenMarkovModelTagger.train(training_notcv)
    results_shuffled_hmm.append(tagger_hmm.evaluate(test_cv))



In [80]:
results_unshuffled_hmm

[0.9795573726779842,
 0.9786173779106642,
 0.9794330916282136,
 0.9794885455514118,
 0.9797124459733616,
 0.9635464733025708,
 0.9629603449494593,
 0.971899852724595,
 0.9655547498187093,
 0.881048086943576]

In [81]:
results_shuffled_hmm

[0.9695986604573282,
 0.9633559339578697,
 0.9702833448514168,
 0.9663281667557456,
 0.9686312295915637,
 0.9685091955203945,
 0.9695559122525415,
 0.9682068673166596,
 0.968098291749744,
 0.967391304347826]

In [83]:
import seaborn as sns
import matplotlib.pyplot as plt
ax.set(ylim=(0.0, 1))

ax = sns.pointplot(list(range(10)),results_unshuffled_tnt,join=False)
ax.set_title('Cross validation 10 Folds TNT - Corpus unshuffled')
ax.set(xlabel="Fold", ylabel = "Accuracy")
ax.set(ylim=(0.75,1))
ax.get_figure().savefig("cv_tnt_unshuffled.png")
plt.clf()

ax = sns.pointplot(list(range(10)),results_unshuffled_hmm,join=False)
ax.set_title('Cross validation 10 Folds HMM - Corpus unshuffled')
ax.set(xlabel="Fold", ylabel = "Accuracy")
ax.set(ylim=(0.85,1))
ax.get_figure().savefig("cv_hmm_unshuffled.png")
plt.clf()

ax = sns.pointplot(list(range(10)),results_shuffled_tnt,join=False)
ax.set_title('Cross validation 10 Folds TNT - Corpus shuffled')
ax.set(xlabel="Fold", ylabel = "Accuracy")
ax.set(ylim=(0.85,1))
ax.get_figure().savefig("cv_tnt_shuffled.png")
ax.set(ylim=(0.0, 1))
plt.clf()

ax = sns.pointplot(list(range(10)),results_shuffled_hmm,join=False)
ax.set_title('Cross validation 10 Folds HMM - Corpus shuffled')
ax.set(xlabel="Fold", ylabel = "Accuracy")
ax.set(ylim=(0.85,1))
ax.get_figure().savefig("cv_hmm_shuffled.png")
plt.clf()

<Figure size 432x288 with 0 Axes>